<a href="https://colab.research.google.com/github/tomonari-masada/course2021-nlp/blob/main/04_word_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **単語ベクトル**
* 前回のように文書をbag-of-wordsモデリングによってベクトルとして表現することは、最近はほとんど行われない。
* まず単語のベクトル表現を得て、それを使って文書のベクトル表現を得る、という手順をとることのほうが多い。
 * こうなったのは、[word2vec](https://arxiv.org/abs/1301.3781)と呼ばれる手法が登場して以降。
  * https://en.wikipedia.org/wiki/Word2vec
* ここでは、spaCyを使って単語ベクトルを得る方法を示す。

## 1. spaCyの単語ベクトル
* 今回は英語テキストのみを扱う。
* 小規模のモデル（名前が__`_sm`__で終わるモデル）は単語ベクトルを含まない。
* 大規模モデルはダウンロードに時間がかかる。
* そのため、ここでは中規模モデルをインストールする。

* 自分のGoogle Driveの適当なディレクトリへにワーキング・ディレクトリを変更する
 * pathは必ず「/content/drive/MyDrive/」で始まるはず。

In [ ]:
%cd /content/drive/MyDrive/2021Courses/NLP/spaCy/

### spaCyの中規模or大規模なモデルをダウンロード
* Google Driveにワーキング・ディレクトリを変更しておけば、ダウンロードしたモデルのファイルが残せる。
* 次からは、あらためてモデル・ファイルをダウンロードしなくても、このファイルをpipでinstallすればよい。

In [ ]:
!wget -nc "https://github.com/explosion/spacy-models/releases/download/en_core_web_md-2.2.5/en_core_web_md-2.2.5.tar.gz"

In [ ]:
!pip install /content/drive/MyDrive/2021Courses/NLP/spaCy/en_core_web_md-2.2.5.tar.gz

* 注：次のセルで「モデルがロードできない」というエラーが出たら、上のColabのメニューの「ランタイム」から「ランタイムを再起動」をクリックして、ランタイムを再起動する。その後、次のセルから実行を再開する（戻らなくてよい）。

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')

## 2. 単語ベクトルを使ってみる

### テキストをtokenizeする

In [ ]:
tokens = nlp("dog cat banana afskfsd")

### トークンを列挙する

In [ ]:
for token in tokens:
  print((f'単語:{token.text}, ベクトルの有無:{token.has_vector},'
        f'ベクトルのL2ノルム:{token.vector_norm:.4f}, ベクトルがOoVか否か:{token.is_oov}'))

### 単語ベクトルの型と中身を確認

* 単語ベクトルはNumPyのndarray

In [ ]:
type(tokens[0].vector)

* 'dog'の単語ベクトルの要素を確認

In [ ]:
tokens[0].vector

* OoV(Out of Vocabulary)の単語ベクトルはゼロベクトル

In [ ]:
print(f'{tokens[3].text}\n{tokens[3].vector}')

* トークン列のベクトルを求めると、OoVのゼロベクトルも含めて平均が計算されるようだ。


In [ ]:
import numpy as np
np.allclose(tokens[0:3].vector * 3, tokens[0:4].vector * 4)

* 多義語であっても単語ベクトルはひとつだけ
 * 意味の数だけ別々のベクトルが用意されていたりはしない。
 * 参考 https://www.youtube.com/watch?v=RB9uDpJPZdc
 * spaCyでは単語ベクトルどうしのコサイン類似度を__`.similarity()`__で計算できる。
 * https://spacy.io/api/token#similarity

* 例：社名のアップルであろうと、りんごのアップルであろうと、単語ベクトルは同一
 * よって、コサイン類似度はぴったり1

In [ ]:
doc = nlp("Apple shares rose on the news. Apple pie is delicious.")
print(doc[0].similarity(doc[7]))

### 文書類似度の計算

In [ ]:
doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")
doc3 = nlp("It's definitely cold outside")

print(doc1.similarity(doc2))
print(doc1.similarity(doc3))
print(doc2.similarity(doc3))

## 3. 単語ベクトルを使った文書分類
* 文書に含まれる単語の単語ベクトルから文書のベクトル表現を得る。
* 文書のベクトル表現を使って2値分類問題を解く。
* sentiment analysisの有名なデータセットであるIMDbを使う。

### IMDbデータセット

* データセットの基本情報
 * Webサイト: https://ai.stanford.edu/~amaas/data/sentiment/
 * 作成者: Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng and Christopher Potts
 * タイトル: Large Movie Review Dataset (aka. IMDb Review Dataset)
 * 公開日: Jun, 2011

### データのロード
* Thincというツールを使う。
 * https://thinc.ai/

In [ ]:
import thinc.extra.datasets
train_data, test_data = thinc.extra.datasets.imdb()
print(f'{len(train_data)} {len(test_data)}')

In [ ]:
train_texts, train_labels = zip(*train_data)
test_texts, test_labels = zip(*test_data)

In [ ]:
train_texts[0]

In [ ]:
print(train_labels)

## 4. fasttextの単語ベクトルを使う
* spaCyの単語ベクトルは、Pythonで実装されているので、遅い。
* fasttextは、C++で実装されているので、速い。

### fasttextのインストール

In [ ]:
!pip install fasttext

### 言語モデルのダウンロード
 * 7GB強のサイズがあるので、非常に時間がかかる。
 * 今回は、諦める。
 * 手順だけを下のセルに示す。

In [ ]:
import fasttext.util
fasttext.util.download_model('en', if_exists='ignore')

### 文書のベクトル化
* モデルをロードしてから、ベクトル化。

In [ ]:
model = fasttext.load_model('cc.en.300.bin')

In [ ]:
import numpy as np

X_train = []
for text in train_texts:
    X_train.append(model.get_sentence_vector(text.replace("\n"," ")))
X_train = np.array(X_train)

X_test = []
for text in test_texts:
    X_test.append(model.get_sentence_vector(text.replace("\n"," ")))
X_test = np.array(X_test)

* ベクトル化した結果をファイルとして保存

In [ ]:
with open('train.npy', 'wb') as f:
    np.save(f, X_train)
with open('test.npy', 'wb') as f:
    np.save(f, X_test)

In [ ]:
with open('train_labels.npy', 'wb') as f:
    np.save(f, np.array(train_labels))
with open('test_labels.npy', 'wb') as f:
    np.save(f, np.array(test_labels))

* 時間がかかるので、事前にベクトル化してある。
 * BlackboardにあるIMDbデータを、あらかじめGoogle Driveに置いておく。

In [ ]:
import numpy as np

PATH = '/content/drive/MyDrive/2021Courses/NLP/'
texts = dict()
labels = dict()
for tag in ['train', 'test']:
  with open(f'{PATH}{tag}.npy', 'rb') as f:
    texts[tag] = np.load(f)
  with open(f'{PATH}{tag}_labels.npy', 'rb') as f:
    labels[tag] = np.load(f)

In [ ]:
texts['train'][0]

## 5. 事前学習済みBERTで文書をベクトル化
* BERTの説明はしない。とりあえず使う。
* BERTを単なるエンコーダとして使う。
* 今回は、下記のspaCy向けのモデルを使う。
 * https://newreleases.io/project/github/explosion/spacy-models/release/en_trf_bertbaseuncased_lg-2.3.0


In [ ]:
!python -m spacy download en_trf_bertbaseuncased_lg

* ここでランタイムを再起動してから、次のセルを実行する。

In [ ]:
import spacy
nlp = spacy.load("en_trf_bertbaseuncased_lg")

In [ ]:
apple1 = nlp("Apple shares rose on the news.")
apple2 = nlp("Apple sold fewer iPhones this quarter.")
apple3 = nlp("Apple pie is delicious.")

# sentence similarity
print(apple1.similarity(apple2)) #0.69861203
print(apple1.similarity(apple3)) #0.5404963

# sentence embeddings
apple1.vector  # or apple1.tensor.sum(axis=0)

* 何の工夫もしないと、単語ベクトルを利用する場合に比べ、非常に時間がかかる。

In [ ]:
import tqdm

X_train = []
for text in tqdm.tqdm(train_texts):
  doc = nlp(text)
  X_train.append(doc.vector)

# 課題４
* 春学期に習った分類手法を使って、IMDbデータセットの感情分析をしてみよう。
 * training set / test setの分割は、そのまま使う。
 * training setをどのように使うかはお任せします。（交差検証など。）
 * test setでの分類性能をArea under the ROC curveで報告。